In [22]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np

from os import path

from plotly import express as px

sys.path.append("../")

from ventilatormodels import SimpleRegressor, LSTMRegressor, VarLSTMRegressor

In [23]:
if not path.isdir("kaggle-data"):
    print("Downloading data...")
    os.mkdir("kaggle-data")
    
    cmds = [
        "cd",
        "kaggle-data",
        "&&",
        "kaggle",
        "competitions",
        "download",
        "-c",
        "ventilator-pressure-prediction",
        "&&",
        "unzip ventilator-pressure-prediction.zip",
    ]
    
    cmd_text = " ".join(cmds)
    
    res = subprocess.run(cmd_text, shell=True, stdout=subprocess.PIPE)
    print("Downloaded and extracted all data!")

In [5]:
training_data = pd.read_csv("kaggle-data/train.csv")
testing_data = pd.read_csv("kaggle-data/test.csv")

In [6]:
training_data.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [20]:
fig = px.line(training_data.head(80*10), x="time_step", y="u_in", color="breath_id")
fig.show()